# Author

- Author: Nicolas Huber
- Mail: nicolassebastian.huber@student.uts.edu.au
- GitHub: HuberNicolas
- Repository: https://github.com/HuberNicolas/python-data-processing-uts

# Source

https://www.kaggle.com/datasets/mexwell/drug-consumption-classification


In [1]:
# Imports

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Constants

PATH = './data/drug_consumption.csv'

In [3]:
# Dev variables
DEV = True
DEV_FRACTION = 0.01

In [4]:
# Explicitly define the order of the target column values
CATEGORIES_ORDER = ['cl0', 'cl1', 'cl2', 'cl3', 'cl4', 'cl5', 'cl6']

In [5]:
# Mapping categories based on source
CATEGORIES = {
    'cl0': 'Never Used',
    'cl1': 'Used over a Decade Ago',
    'cl2': 'Used in Last Decade',
    'cl3': 'Used in Last Year',
    'cl4': 'Used in Last Month',
    'cl5': 'Used in Last Week',
    'cl6': 'Used in Last Day'
}

In [6]:
COLUMNS_TYPE = {
    'ID': int,
    'Age': float,
    'Gender': float,
    'Education': float,
    'Country': float,
    'Ethnicity': float,
    'Nscore': float,
    'Escore': float,
    'Oscore': float,
    'Ascore': float,
    'Cscore': float,
    'Impulsive': float,
    'SS': float,
    'Alcohol': str,
    'Amphet': str,
    'Amyl': str,
    'Benzos': str,
    'Caff': str,
    'Cannabis': str,
    'Choc': str,
    'Coke': str,
    'Crack': str,
    'Ecstasy': str,
    'Heroin': str,
    'Ketamine': str,
    'Legalh': str,
    'LSD': str,
    'Meth': str,
    'Mushrooms': str,
    'Nicotine': str,
    'Semer': str,
    'VSA': str
}

In [7]:
# Define columns for better clarity
RENAME_COLUMNS = {
    # Big Five
    # https://en.wikipedia.org/wiki/Big_Five_personality_traits
    'Nscore': 'Neuroticism_score',
    'Escore': 'Extraversion_score',
    'Oscore': 'Openness_score',
    'Ascore': 'Agreeableness_score',
    'Cscore': 'Conscientiousness_score',

    'Impulsive': 'Impulsive_score',
    'SS': 'Sensation_seeing_score'
}